# Xử lý dữ liệu

Xử lý dữ liệu đơn giản

- Tất cả các cột category -> onehot
- Các cột numeric -> fillna mean

In [1]:
import pandas as pd

df = pd.read_csv('data/preprocessed_data.csv')

In [6]:
df.drop(
    [
        'url',
        'date',
        'out_color',
        'in_color',
        'num_door',
        'num_seat',
        'gear_type',
        'drive_type',
        'cylinder_capacity'
    ],
    axis=1,
    inplace=True
)

In [12]:
df

,brand,serie,model_year,price_vnd,origin,status,category,used_distance
0,Ford,Focus,2018.0,478,Lắp ráp trong nước,Xe đã dùng,Sedan,0
1,Mazda,CX5,2021.0,999,Lắp ráp trong nước,Xe mới,SUV,0
2,Kia,Cerato,2018.0,510,Lắp ráp trong nước,Xe đã dùng,Sedan,2
3,Isuzu,Dmax,2021.0,580,Nhập khẩu,Xe mới,Bán tải / Pickup,0
4,Kia,Sorento,2021.0,1230,Lắp ráp trong nước,Xe đã dùng,SUV,0
...,...,...,...,...,...,...,...,...
41608,Hãng khác,NaN,NaN,65,Lắp ráp trong nước,Xe đã dùng,Truck,0
41609,Toyota,Corolla,2002.0,155,Lắp ráp trong nước,Xe đã dùng,Sedan,0
41610,Ford,Ranger,2016.0,675,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,0
41611,Chrysler,300C,2008.0,555,Nhập khẩu,Xe đã dùng,Sedan,58000


In [10]:
cat_columns_df = df[['brand', 'serie', 'origin', 'status', 'category']]

In [14]:
numeric_columns_df = df[['model_year', 'price_vnd', 'used_distance']]

In [19]:
df = pd.concat([numeric_columns_df, pd.get_dummies(cat_columns_df)], axis=1)

In [20]:
df

,model_year,price_vnd,used_distance,brand_Acura,brand_Alfa Romeo,brand_Asia,brand_Aston Martin,brand_Audi,brand_BMW,brand_BYD,...,category_Bán tải / Pickup,category_Convertible/Cabriolet,category_Coupe,category_Crossover,category_Hatchback,category_SUV,category_Sedan,category_Truck,category_Van/Minivan,category_Wagon
0,2018.0,478,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2021.0,999,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2018.0,510,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2021.0,580,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2021.0,1230,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41608,NaN,65,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
41609,2002.0,155,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
41610,2016.0,675,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
41611,2008.0,555,58000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [32]:
df['model_year'] = df['model_year'].fillna(value=df['model_year'].mean())

# Train model

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['price_vnd'], axis=1), 
    df['price_vnd'],
    test_size=0.2
)

In [38]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [39]:
model.fit(X_train, y_train)

LinearRegression()

# Model evaluation

Sử dụng hàm này để tính kết quả. Kết quả sẽ tự động ghi lên airtable

In [41]:
from helpers.evaluation import get_summary_result

In [42]:
get_summary_result(model, X_test, y_test)

{'Model': 'LinearRegression', 'Hyperparams': "{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': 'deprecated', 'positive': False}", 'R2': 0.7371649601383088, 'RMSE': 881.1756871404298, 'MAE': 309.61302713860795, 'MAPE': 3.410925844183934, 'Time': '00:00:52 22-03-2022'}
